In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
x = [[1, 2], [3, 4], [5, 6], [7, 8]]
y = [[3], [7], [11], [15]]

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

X = torch.tensor(x).float().to(device)
Y = torch.tensor(y).float().to(device)

In [4]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x.detach().clone()
        self.y = y.detach().clone()
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

In [5]:
dataset = MyDataset(X, Y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [6]:
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2, 8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8, 1)
    
    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x

my_net = MyNeuralNet().to(device)

In [7]:
def my_mean_squared_error(y_predict, y):
    return ((y_predict - y) ** 2).mean()

## Compare our function with `MSELoss`

In [8]:
loss_function = nn.MSELoss()
loss_value = loss_function(my_net(X), Y)
loss_value

tensor(79.7226, device='cuda:0', grad_fn=<MseLossBackward>)

In [9]:
my_mean_squared_error(my_net(X), Y)

tensor(79.7226, device='cuda:0', grad_fn=<MeanBackward0>)

# Fetching the values of intermediate layers

In [10]:
input_to_hidden = my_net.input_to_hidden_layer(X)
hidden_activation = my_net.hidden_layer_activation(input_to_hidden)
hidden_activation

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9132, 0.0000, 0.3910],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.4359, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 3.9587, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 5.4814, 0.0606, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)